# Task1: Write a Python code for this problem using the relevant libraries. 

We create a Tree, definining its node. By default we assume that beta = infinite and alfa=- infinite. The algoritm of minmax we're implementing is quite simple. We recursively go down the stack and compute the values of alpha e beta.

In [1]:
class TreeNode:
    def __init__(self,value = None):
        self.alfa = float('-inf')
        self.beta = float('inf')
        self.value = value
        self.children = []
        self.parent = None
    def is_leaf(self):
        return len(self.children) == 0

This is the minmax algorithm. We visit in DFS the tree defined above and compute the max. Typically in games each minmax level 
changes. The max become min level as soon we go down and viceverse.

In [2]:
def minimax(node, depth, isMaxLevel, alpha, beta):
    if depth == 0 or node.is_leaf():
        return node.value
    # maxPlayer determine if it is a max level
    if isMaxLevel:
        max_eval = float('-inf')
        for child in node.children:
            current_eval = minimax(child, depth - 1, False, alpha, beta)
            max_eval = max(max_eval, current_eval)
            alpha = max(alpha, max_eval)
            child.alpha = alpha
            print(f'Value = {child.value}, Alpha={alpha} Beta={beta}')
            if beta <= alpha:
                print(f'Pruning : {beta}<={alpha}')
                break
        return max_eval
    else:
        minval = float('inf')
        for child in node.children:
            value = minimax(child, depth - 1, True, alpha, beta)
            minval = min(minval, value)
            beta = min(beta, minval)
            child.beta = beta
            print(f'Value = {child.value}, Alpha={alpha} Beta={beta}')
            if beta <= alpha:
                print(f'Pruning : {beta}<={alpha}')
                break
        return minval


In [3]:

def build_sample_tree() -> TreeNode:
    root = TreeNode()
    left_tree = TreeNode(5.0)
    left_tree.parent = root
    right_tree = TreeNode()
    right_tree.parent = root

    right_tree_subtree_right = TreeNode()
    right_tree_subtree_left = TreeNode()

    right_tree_subtree_right.parent = right_tree
    right_tree_subtree_left.parent = right_tree

    right_tree_subtree_left_right_right = TreeNode()
    right_tree_subtree_left_right_right.children = [TreeNode(4.0), TreeNode(2.0)]
    right_tree_subtree_left.children = [TreeNode(1.0), right_tree_subtree_left_right_right]
    
    right_tree_subtree_right = TreeNode()
    right_tree_subtree_right_right = TreeNode()
    right_tree_subtree_right.children = [ TreeNode(5.0), right_tree_subtree_right_right]
    right_tree_subtree_right_right.children = [ TreeNode(4.0), TreeNode(3.0)]
    # add subtree
    right_tree.children = [ right_tree_subtree_left, right_tree_subtree_right]
    root.children = [left_tree, right_tree]
    return root
    


if __name__=="__main__":    
    root = build_sample_tree()
    best_value = minimax(root, 6, True, float('-inf'), float('+inf'))
    print(f"Goal {best_value}")

Value = 5.0, Alpha=5.0 Beta=inf
Value = 1.0, Alpha=5.0 Beta=inf
Value = 4.0, Alpha=5.0 Beta=4.0
Pruning : 4.0<=5.0
Value = None, Alpha=5.0 Beta=inf
Value = None, Alpha=5.0 Beta=4.0
Pruning : 4.0<=5.0
Value = None, Alpha=5.0 Beta=inf
Goal 5.0


# Task2 :Create a Jupyter Notebook to solve this constraint satisfaction problem using appropriate Python libraries. Use markdown cells to provide a description of what your code is doing.

Assumptions:
   - We had fix Anna to the last 2 shifts to reduce the space of the solutins.
   - Each shift can be covered only by one staff member.
   - Each staff member should cover as equal as several shifts as possible over the four days.

We provide comments inside the code and avoid to split the function.

In [4]:
!pip install ortools


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [5]:
from ortools.sat.python import cp_model

Creare the CSP model using orttools

In [6]:
model = cp_model.CpModel()

Set the staff and mapping to daily slot

In [7]:
staff = ['Mark', 'Judy', 'Colm', 'Dave', 'Jane', 'Anne', 'Gavin', 'Tanya']
mapped_hours = { key: value for key, value in zip(range(9,19), range(0,10)) }
staff_time_slots = {'Mark': (9, 14), 
                        'Judy': (10, 15),
                        'Colm': (13, 15),
                        'Dave': (11, 12),
                        'Jane': (10, 13),
                        'Anne': (14, 18),
                        'Gavin': (13, 17),
                        'Tanya': (9, 15)}
    
num_days = 4
hours_per_day = 9  # From 9:00 to 18:00


Now we need to create shift variables. Each bool shift variable for each shift and member of the staff

In [8]:
# Create shift variables
shifts = {}
for day in range(num_days):
    for hour in range(hours_per_day):
        for s in range(len(staff)):
            shifts[(day, hour, s)] = model.new_bool_var(f'shift_{day}_{hour}_{staff[s]}')


Each slot is covered by exactly one staff member.

In [9]:
for day in range(num_days):
    for hour in range(hours_per_day):
        model.add_exactly_one(shifts[(day, hour, s)] for s in range(len(staff)))
    
    

This an assumption that I am doing: Everyone should work.

In [10]:
# everyone should work.
for s in range(len(staff)):
    for day in range(num_days):
        shift_sum = sum(shifts[(day, hour, s)] for hour in range(hours_per_day)) >= 1
        model.add(shift_sum)


In [11]:
# Each member the staff except Anne, will work just one shift
# Annie will be pinned in 2 shift 3 - 4 p.m and 5-6 p.m. They are not consecutive.
for staff_index in  range(len(staff)):
      anne_index = staff.index('Anne')
      if anne_index != staff_index:
         for day in range(num_days):
            model.add_at_most_one(shifts[(day,hour,staff_index)] for hour in range(hours_per_day))

We pin Anne at 3 p.m and 5 p.m. All the other will have the same number of hours.

In [12]:
# i pin Anne everyday at 3 p.m and 5 p.m. This allows her to stay in different slots non consecutive and reduce the space of solutions.
# Anne is the only one that has a fixed slot.
anne_index = staff.index('Anne')
for day in range(num_days):
    model.add(sum(shifts[(day, hour, anne_index)] for hour in range(hours_per_day)) <= 2)
    model.add(shifts[(day, mapped_hours[15], anne_index)] == 1)  #
   

Id we would want instead not allow two consecutive hours, we could relax the at most one hour and avoid the particular case of Anne.
``` python 
for s in range(len(staff)):
    for day in range(num_days):
        for hour in range(hours_per_day - 1):
            model.add_implication(shifts[(day, hour, s)], shifts[(day, hour + 1, s)].Not())
```
With this binding\.

In [13]:
# Each staff member can only work within their available times
for name in staff:
    staff_index = staff.index(name)
    for day in range(num_days):
        for hour in range(hours_per_day):
            start = staff_time_slots[name][0]
            end = staff_time_slots[name][1]
            if hour < mapped_hours[start] or hour >= mapped_hours[end]:  
                model.add(shifts[(day, hour, staff_index)] == 0)    


In [14]:
# Try to distribute the shifts evenly between all except annie
min_shift_per_staff = (hours_per_day * num_days) // (len(staff) - 1)
if hours_per_day * num_days % (len(staff) -1) == 0:
    max_shifts_per_staff = min_shift_per_staff
else:
    max_shifts_per_staff = min_shift_per_staff + 1
    
for staff_index in range(len(staff)):
    annie_index = staff.index('Anne')
    if staff_index != annie_index:
        shifts_worked = []
        for day in range(num_days):
            for hour in range(hours_per_day):
                shifts_worked.append(shifts[(day, hour, staff_index)])
        model.add(max_shifts_per_staff <= sum(shifts_worked))
        model.add(sum(shifts_worked) <= max_shifts_per_staff)
    

In [15]:
# Solve the model
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
solver.parameters.enumerate_all_solutions = True

In [16]:
class ReceptionistShiftPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, shifts, num_days, hours_per_day, staff):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_days = num_days
        self._hours_per_day = hours_per_day
        self._staff = staff
        self.solution_count = 0
        
    def OnSolutionCallback(self):
        self.solution_count += 1
        print(f'Solution {self.solution_count}')


In [17]:
# Create the solution printer and solve
solution_printer = ReceptionistShiftPrinter(shifts, num_days, hours_per_day, staff)
solver.parameters.max_time_in_seconds = 10.0
solver.Solve(model, solution_printer)
print(f'Number of solutions found: {solution_printer.solution_count}')


Number of solutions found: 0
